# Deeper Dive on Process Analysis

### Users
To analyze users associated with processes you can use the plugin `windows.getsids.GetSIDs`, SIDs standing for security identifier. You should see SIDs for local and domain users

In [1]:
python ../volatility3/vol.py -q -f E:\APT.img windows.getsids.GetSIDs

The below script needs to be able to find the current output cell; this is an easy method to get it.

Volatility 3 Framework 1.2.1

PID	Process	SID	Name

4	System	S-1-5-18	Local System
4	System	S-1-5-32-544	Administrators
4	System	S-1-1-0	Everyone
4	System	S-1-5-11	Authenticated Users
564	smss.exe	S-1-5-18	Local System
564	smss.exe	S-1-5-32-544	Administrators
564	smss.exe	S-1-1-0	Everyone
564	smss.exe	S-1-5-11	Authenticated Users
636	csrss.exe	S-1-5-18	Local System
636	csrss.exe	S-1-5-32-544	Administrators
636	csrss.exe	S-1-1-0	Everyone
636	csrss.exe	S-1-5-11	Authenticated Users
660	winlogon.exe	S-1-5-18	Local System
660	winlogon.exe	S-1-5-32-544	Administrators
660	winlogon.exe	S-1-1-0	Everyone
660	winlogon.exe	S-1-5-11	Authenticated Users
704	services.exe	S-1-5-18	Local System
704	services.exe	S-1-5-32-544	Administrators
704	services.exe	S-1-1-0	Everyone
704	services.exe	S-1-5-11	Authenticated Users
716	lsass.exe	S-1-5-18	Local System
716	lsass.exe	S-1-5-32-544	Administrators
716	lsass.exe	S-1-1-0	Everyone
716	lsass.exe	S-1-5-11	Authenticated Users
872	vmacthlp.exe	S-1-5-18	Local Syst

Reviewing all the SIDs can be daunting so try to focus just on the process we identified as supicoius and parent processes in this case PID (796)

### Privilleges

You can also see what privilleges are associated with each user if you do not have access to that infomration in other formats (like if you don't have access to the affected domain). You can enumerate privilleges with the privs command as shown below:

In [2]:
python ../volatility3/vol.py -q -f E:\APT.img windows.privileges.Privs

Volatility 3 Framework 1.2.1

PID	Process	Value	Privilege	Attributes	Description

4	System	7	SeTcbPrivilege	Present,Enabled,Default	Act as part of the operating system
4	System	2	SeCreateTokenPrivilege	Present	Create a token object
4	System	9	SeTakeOwnershipPrivilege	Present	Take ownership of files/objects
4	System	15	SeCreatePagefilePrivilege	Present,Enabled,Default	Create a pagefile
4	System	4	SeLockMemoryPrivilege	Present,Enabled,Default	Lock pages in memory
4	System	3	SeAssignPrimaryTokenPrivilege	Present	Replace a process-level token
4	System	5	SeIncreaseQuotaPrivilege	Present	Increase quotas
4	System	14	SeIncreaseBasePriorityPrivilege	Present,Enabled,Default	Increase scheduling priority
4	System	16	SeCreatePermanentPrivilege	Present,Enabled,Default	Create permanent shared objects
4	System	20	SeDebugPrivilege	Present,Enabled,Default	Debug programs
4	System	21	SeAuditPrivilege	Present,Enabled,Default	Generate security audits
4	System	8	SeSecurityPrivilege	Present	Manage auditing an

704	services.exe	9	SeTakeOwnershipPrivilege	Present	Take ownership of files/objects
704	services.exe	15	SeCreatePagefilePrivilege	Present,Enabled,Default	Create a pagefile
704	services.exe	4	SeLockMemoryPrivilege	Present,Enabled,Default	Lock pages in memory
704	services.exe	3	SeAssignPrimaryTokenPrivilege	Present	Replace a process-level token
704	services.exe	5	SeIncreaseQuotaPrivilege	Present	Increase quotas
704	services.exe	14	SeIncreaseBasePriorityPrivilege	Present,Enabled,Default	Increase scheduling priority
704	services.exe	16	SeCreatePermanentPrivilege	Present,Enabled,Default	Create permanent shared objects
704	services.exe	20	SeDebugPrivilege	Present,Enabled,Default	Debug programs
704	services.exe	21	SeAuditPrivilege	Present,Enabled,Default	Generate security audits
704	services.exe	8	SeSecurityPrivilege	Present	Manage auditing and security log
704	services.exe	22	SeSystemEnvironmentPrivilege	Present	Edit firmware environment values
704	services.exe	23	SeChangeNotifyPrivilege	Pre

968	svchost.exe	3	SeAssignPrimaryTokenPrivilege	Present	Replace a process-level token
968	svchost.exe	23	SeChangeNotifyPrivilege	Present,Enabled,Default	Receive notifications of changes to files or directories
968	svchost.exe	19	SeShutdownPrivilege	Present	Shut down the system
968	svchost.exe	25	SeUndockPrivilege	Present	Remove computer from docking station
968	svchost.exe	29	SeImpersonatePrivilege	Present,Enabled,Default	Impersonate a client after authentication
968	svchost.exe	30	SeCreateGlobalPrivilege	Present,Enabled,Default	Create global objects
1088	svchost.exe	7	SeTcbPrivilege	Present,Enabled,Default	Act as part of the operating system
1088	svchost.exe	2	SeCreateTokenPrivilege	Present,Enabled	Create a token object
1088	svchost.exe	9	SeTakeOwnershipPrivilege	Present,Enabled	Take ownership of files/objects
1088	svchost.exe	15	SeCreatePagefilePrivilege	Present,Enabled,Default	Create a pagefile
1088	svchost.exe	4	SeLockMemoryPrivilege	Present,Enabled,Default	Lock pages in memory
108

1984	VMwareTray.exe	18	SeRestorePrivilege	Present	Restore files and directories
1984	VMwareTray.exe	12	SeSystemtimePrivilege	Present	Change the system time
1984	VMwareTray.exe	19	SeShutdownPrivilege	Present	Shut down the system
1984	VMwareTray.exe	24	SeRemoteShutdownPrivilege	Present	Force shutdown from a remote system
1984	VMwareTray.exe	9	SeTakeOwnershipPrivilege	Present	Take ownership of files/objects
1984	VMwareTray.exe	20	SeDebugPrivilege	Present	Debug programs
1984	VMwareTray.exe	22	SeSystemEnvironmentPrivilege	Present	Edit firmware environment values
1984	VMwareTray.exe	11	SeSystemProfilePrivilege	Present	Profile system performance
1984	VMwareTray.exe	13	SeProfileSingleProcessPrivilege	Present	Profile a single process
1984	VMwareTray.exe	14	SeIncreaseBasePriorityPrivilege	Present	Increase scheduling priority
1984	VMwareTray.exe	10	SeLoadDriverPrivilege	Present,Enabled	Load and unload device drivers
1984	VMwareTray.exe	15	SeCreatePagefilePrivilege	Present	Create a pagefile
1984	V

1260	wscntfy.exe	9	SeTakeOwnershipPrivilege	Present	Take ownership of files/objects
1260	wscntfy.exe	20	SeDebugPrivilege	Present	Debug programs
1260	wscntfy.exe	22	SeSystemEnvironmentPrivilege	Present	Edit firmware environment values
1260	wscntfy.exe	11	SeSystemProfilePrivilege	Present	Profile system performance
1260	wscntfy.exe	13	SeProfileSingleProcessPrivilege	Present	Profile a single process
1260	wscntfy.exe	14	SeIncreaseBasePriorityPrivilege	Present	Increase scheduling priority
1260	wscntfy.exe	10	SeLoadDriverPrivilege	Present	Load and unload device drivers
1260	wscntfy.exe	15	SeCreatePagefilePrivilege	Present	Create a pagefile
1260	wscntfy.exe	5	SeIncreaseQuotaPrivilege	Present	Increase quotas
1260	wscntfy.exe	25	SeUndockPrivilege	Present	Remove computer from docking station
1260	wscntfy.exe	28	SeManageVolumePrivilege	Present	Manage the files on a volume
1260	wscntfy.exe	29	SeImpersonatePrivilege	Present,Enabled,Default	Impersonate a client after authentication
1260	wscntfy.exe	3

456	MIRAgent.exe	28	SeManageVolumePrivilege	Present	Manage the files on a volume
456	MIRAgent.exe	29	SeImpersonatePrivilege	Present,Enabled,Default	Impersonate a client after authentication
456	MIRAgent.exe	30	SeCreateGlobalPrivilege	Present,Enabled,Default	Create global objects


### Loaded DLL's

In [3]:
python ../volatility3/vol.py -q -f E:\APT.img windows.dlllist.DllList --pid=796

Volatility 3 Framework 1.2.1

PID	Process	Base	Size	Name	Path	LoadTime	File output

796	iexplore.exe	0x400000	0x9b000	iexplore.exe	C:\Program Files\Internet Explorer\iexplore.exe	N/A	Disabled
796	iexplore.exe	0x7c900000	0xaf000	ntdll.dll	C:\WINDOWS\system32\ntdll.dll	N/A	Disabled
796	iexplore.exe	0x7c800000	0xf6000	kernel32.dll	C:\WINDOWS\system32\kernel32.dll	N/A	Disabled
796	iexplore.exe	0x77dd0000	0x9b000	ADVAPI32.dll	C:\WINDOWS\system32\ADVAPI32.dll	N/A	Disabled
796	iexplore.exe	0x77e70000	0x92000	RPCRT4.dll	C:\WINDOWS\system32\RPCRT4.dll	N/A	Disabled
796	iexplore.exe	0x77fe0000	0x11000	Secur32.dll	C:\WINDOWS\system32\Secur32.dll	N/A	Disabled
796	iexplore.exe	0x77f10000	0x49000	GDI32.dll	C:\WINDOWS\system32\GDI32.dll	N/A	Disabled
796	iexplore.exe	0x7e410000	0x91000	USER32.dll	C:\WINDOWS\system32\USER32.dll	N/A	Disabled
796	iexplore.exe	0x77c10000	0x58000	msvcrt.dll	C:\WINDOWS\system32\msvcrt.dll	N/A	Disabled
796	iexplore.exe	0x77f60000	0x76000	SHLWAPI.dll	C:\WINDOWS\system32\SHLWAP

In [8]:
python ../volatility3/vol.py -q -r csv -f E:\APT.img windows.dlllist.DllList --pid=796 | csvcut -c 2,6,7 | csvlook

Volatility 3 Framework 1.2.1
| PID | Name         | Path                                                                                                              |
| --- | ------------ | ----------------------------------------------------------------------------------------------------------------- |
| 796 | iexplore.exe | C:\Program Files\Internet Explorer\iexplore.exe                                                                   |
| 796 | ntdll.dll    | C:\WINDOWS\system32\ntdll.dll                                                                                     |
| 796 | kernel32.dll | C:\WINDOWS\system32\kernel32.dll                                                                                  |
| 796 | ADVAPI32.dll | C:\WINDOWS\system32\ADVAPI32.dll                                                                                  |
| 796 | RPCRT4.dll   | C:\WINDOWS\system32\RPCRT4.dll                                                                                    

In [9]:
python ../volatility3/vol.py -q -f E:\APT.img windows.handles.Handles --pid=796

Volatility 3 Framework 1.2.1

PID	Process	Offset	HandleValue	Type	GrantedAccess	Name

796	iexplore.exe	0x821ace98	0xc	File	0x100020	\Device\HarddiskVolume1\WINDOWS\system32
796	iexplore.exe	0x82189028	0x578	File	0x12019f	\Device\NamedPipe\ROUTER
796	iexplore.exe	0x822eff10	0x57c	Event	0x1f0003	
796	iexplore.exe	0x81f229d0	0x580	Semaphore	0x100003	
796	iexplore.exe	0x81f22a08	0x584	Semaphore	0x100003	
796	iexplore.exe	0x81f30838	0x588	Semaphore	0x100003	
796	iexplore.exe	0x822eae40	0x58c	Semaphore	0x100003	
796	iexplore.exe	0xe1c29c90	0x590	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
796	iexplore.exe	0xe1117fb8	0x594	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
796	iexplore.exe	0xe1c29c28	0x598	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
796	iexplore.exe	0xe1e9f728	0x59c	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\LINKAGE
796	iexplore.exe	0x821aaeb8	0x5a0	Semaphore	0x100003	
796	iexplore.exe	0x822

796	iexplore.exe	0xe1c5ade8	0x774	Key	0xf003f	MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\NAMESPACE_CATALOG5
796	iexplore.exe	0x81e7d5b8	0x778	Event	0x1f0003	
796	iexplore.exe	0xe111e2a8	0x77c	Key	0xf003f	MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\PROTOCOL_CATALOG9
796	iexplore.exe	0x822ed538	0x780	Event	0x1f0003	
796	iexplore.exe	0x81dd17a8	0x784	Event	0x1f0003	
796	iexplore.exe	0x81d298c8	0x788	File	0x100020	\Device\HarddiskVolume1\WINDOWS\WinSxS\x86_Microsoft.Windows.Common-Controls_6595b64144ccf1df_6.0.2600.5512_x-ww_35d4ce83
796	iexplore.exe	0x82178de8	0x78c	Event	0x100003	
796	iexplore.exe	0x82070c98	0x790	WmiGuid	0xa84	
796	iexplore.exe	0x81f2a3b8	0x794	Event	0x1f0003	
796	iexplore.exe	0x81fb7110	0x798	Event	0x1f0003	
796	iexplore.exe	0x81dbdda0	0x79c	Process	0x1f0fff	iexplore.exe Pid 796
796	iexplore.exe	0x81e72220	0x7a0	Event	0x1f0003	
796	iexplore.exe	0x81f224c0	0x7a4	File	0x12019f	\Device\WMIDataDevice
796	iexplore.exe	0x81d1cf48	0x7a8	WmiGuid	0x

You used to be able to limit what handles are enumerated by using the `-t ObjectType` but you can use csvkit to do the same with the command line below:

In [18]:
python ../volatility3/vol.py -q -r csv -f E:\APT.img windows.handles.Handles --pid=796 | csvcut -c 2,3,5,6,8 | csvgrep -c 4 -r "File|Key"

Volatility 3 Framework 1.2.1
PID,Process,HandleValue,Type,Name
796,iexplore.exe,0xc,File,\Device\HarddiskVolume1\WINDOWS\system32
796,iexplore.exe,0x578,File,\Device\NamedPipe\ROUTER
796,iexplore.exe,0x590,Key,MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
796,iexplore.exe,0x594,Key,MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
796,iexplore.exe,0x598,Key,MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
796,iexplore.exe,0x59c,Key,MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\LINKAGE
796,iexplore.exe,0x5a8,File,\Device\Ip
796,iexplore.exe,0x5ac,File,\Device\Ip
796,iexplore.exe,0x5b0,File,\Device\Ip
796,iexplore.exe,0x5b4,File,\Device\Tcp
796,iexplore.exe,0x5b8,File,\Device\Tcp
796,iexplore.exe,0x5bc,File,\Device\NamedPipe\ROUTER
796,iexplore.exe,0x5c4,File,\Device\Afd\AsyncConnectHlp
796,iexplore.exe,0x5f8,Key,USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\INTERNET SETTINGS\ZONEMAP
796,iexplore.exe,0x5fc,Key,MACHINE\SOFTWARE
796,iexplore.exe,0x600

## Command Line History
### CmdLine
You can see the command line history by enumerating COMMAND_HISTORY objects from memory. There are different plugins that can parse these data points. This enumeration works by looking for a known constant called `(MaxHistory)` which is a parameter that stores the max number of commands to keep in a buffer. 

In [19]:
python ../volatility3/vol.py -q -r csv -f E:\APT.img windows.cmdline.CmdLine

Volatility 3 Framework 1.2.1
"TreeDepth","PID","Process","Args"
0,4,"System","Required memory at 0x10 is not valid (process exited?)"
0,564,"smss.exe","\SystemRoot\System32\smss.exe"
0,636,"csrss.exe","C:\WINDOWS\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,3072,512 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=winsrv:ConServerDllInitialization,2 ProfileControl=Off MaxRequestThreads=16"
0,660,"winlogon.exe","winlogon.exe"
0,704,"services.exe","C:\WINDOWS\system32\services.exe"
0,716,"lsass.exe","C:\WINDOWS\system32\lsass.exe"
0,872,"vmacthlp.exe",""C:\Program Files\VMware\VMware Tools\vmacthlp.exe""
0,884,"svchost.exe","C:\WINDOWS\system32\svchost -k DcomLaunch"
0,968,"svchost.exe","C:\WINDOWS\system32\svchost -k rpcss"
0,1088,"svchost.exe","C:\WINDOWS\System32\svchost.exe -k netsvcs"
0,1140,"svchost.exe","C:\WINDOWS\system32\svchost.exe -k NetworkService"
0,1212,"svchost.exe","C:\WINDOWS\system32\svchost